In [1]:
%cd drive/MyDrive/kaggle/JaneStreet

/content/drive/MyDrive/kaggle/JaneStreet


In [2]:
# !git clone https://github.com/keras-team/keras-tuner
!cd keras-tuner && pip -q install .

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.model_selection import GroupKFold

from tqdm import tqdm
from random import choices
import random


import kerastuner as kt

def set_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

In [4]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]
            
class CVTuner(kt.engine.tuner.Tuner):
    def run_trial(self, trial, X, y, splits, batch_size=32, epochs=1,callbacks=None):
        val_losses = []
        for train_indices, test_indices in splits:
            X_train, X_test = [x[train_indices] for x in X], [x[test_indices] for x in X]
            y_train, y_test = [a[train_indices] for a in y], [a[test_indices] for a in y]
            if len(X_train) < 2:
                X_train = X_train[0]
                X_test = X_test[0]
            if len(y_train) < 2:
                y_train = y_train[0]
                y_test = y_test[0]
            
            model = self.hypermodel.build(trial.hyperparameters)
            hist = model.fit(X_train,y_train,
                      validation_data=(X_test,y_test),
                      epochs=epochs,
                        batch_size=batch_size,
                      callbacks=callbacks)
            
            val_losses.append([hist.history[k][-1] for k in hist.history])
        val_losses = np.asarray(val_losses)
        self.oracle.update_trial(trial.trial_id, {k:np.mean(val_losses[:,i]) for i,k in enumerate(hist.history.keys())})
        self.save_model(trial.trial_id, model)

# From https://medium.com/@micwurm/using-tensorflow-lite-to-speed-up-predictions-a3954886eb98

class LiteModel:
    
    @classmethod
    def from_file(cls, model_path):
        return LiteModel(tf.lite.Interpreter(model_path=model_path))
    
    @classmethod
    def from_keras_model(cls, kmodel):
        converter = tf.lite.TFLiteConverter.from_keras_model(kmodel)
        tflite_model = converter.convert()
        return LiteModel(tf.lite.Interpreter(model_content=tflite_model))
    
    def __init__(self, interpreter):
        self.interpreter = interpreter
        self.interpreter.allocate_tensors()
        input_det = self.interpreter.get_input_details()[0]
        output_det = self.interpreter.get_output_details()[0]
        self.input_index = input_det["index"]
        self.output_index = output_det["index"]
        self.input_shape = input_det["shape"]
        self.output_shape = output_det["shape"]
        self.input_dtype = input_det["dtype"]
        self.output_dtype = output_det["dtype"]
        
    def predict(self, inp):
        inp = inp.astype(self.input_dtype)
        count = inp.shape[0]
        out = np.zeros((count, self.output_shape[1]), dtype=self.output_dtype)
        for i in range(count):
            self.interpreter.set_tensor(self.input_index, inp[i:i+1])
            self.interpreter.invoke()
            out[i] = self.interpreter.get_tensor(self.output_index)[0]
        return out
    
    def predict_single(self, inp):
        """ Like predict(), but only for a single record. The input data can be a Python list. """
        inp = np.array([inp], dtype=self.input_dtype)
        self.interpreter.set_tensor(self.input_index, inp)
        self.interpreter.invoke()
        out = self.interpreter.get_tensor(self.output_index)
        return out[0]

import shutil
def del_his(n, v):
    try:
        shutil.rmtree(f'models_v{v}/jane_street_{n}')
        os.remove(f"models_v{v}/best_hp_{n}.pkl")
        # os.remove(f"models_v{v}/model_{n}_0.hdf5")
        os.remove(f"models_v{v}/model_{n}_0_finetune.hdf5")
        # os.remove(f"models_v{v}/model_{n}_1.hdf5")
        os.remove(f"models_v{v}/model_{n}_1_finetune.hdf5")
        # os.remove(f"models_v{v}/model_{n}_2.hdf5")
        os.remove(f"models_v{v}/model_{n}_2_finetune.hdf5")
        # os.remove(f"models_v{v}/model_{n}_3.hdf5")
        os.remove(f"models_v{v}/model_{n}_3_finetune.hdf5")
        os.remove(f"models_v{v}/model_{n}_4.hdf5")
        os.remove(f"models_v{v}/model_{n}_4_finetune.hdf5")
    except:
        pass

In [5]:
train = pd.read_csv('./train.csv')
# train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

train.fillna(train.mean(),inplace=True)
features = [c for c in train.columns if 'feature' in c]

from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
# embedding_matrix = pd.read_csv('features.csv').iloc[0:,1:].values.astype(np.float32)
# X = train[features].values
X = scaler.fit_transform(train[features].values)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

f_mean = np.mean(train[features[1:]].values,axis=0)

feature = pd.read_csv('./features.csv')
feature_list = [np.where(feature.values[:, i]==True)[0] for i in range(1, len(feature.values[0]))]

In [6]:
def tag_layer(x):
    x = BatchNormalization()(x)
    # x = GaussianNoise(noise)(x)
    x = Dense(32, activation="sigmoid")(x)
    x = Dropout(0.75)(x)
    return x

## v2
def create_model(input_dim,output_dim):
    inputs = Input(input_dim)
    inputs2 = tf.keras.backend.square(inputs)
    tags = []
    for f in feature_list:
        tags.append(tag_layer(tf.gather(inputs, tf.constant(f), axis=-1)))
    Tags = Concatenate()(tags)
    x = Concatenate(axis=-1)([inputs, inputs2, Tags])

    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    for i in range(3):
        x = Dense(256)(x)
        x = BatchNormalization()(x)
        x = Lambda(tf.keras.activations.swish)(x)
        x = Dropout(0.5)(x)

    x = tf.keras.layers.Dense(output_dim)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = Model(inputs=inputs,outputs=out)
    model.compile(
        optimizer=tfa.optimizers.RectifiedAdam(1e-3),
        loss=BinaryCrossentropy(label_smoothing=1e-3),
        # loss="mse",
        metrics=tf.keras.metrics.AUC(name = 'auc')
        )
    return model
v = 2
try:
    os.mkdir(f"models_v{v}")
except:
    pass

In [ ]:
def tag_layer(x):
    x = BatchNormalization()(x)
    # x = GaussianNoise(noise)(x)
    x = Dense(128, activation="sigmoid")(x)
    x = Dropout(0.8)(x)
    return x

## v6
def create_model(input_dim,output_dim):
    inputs = Input(input_dim)
    inputs2 = tf.keras.backend.square(inputs)
    tags = []
    for f in feature_list:
        tags.append(tag_layer(tf.gather(inputs, tf.constant(f), axis=-1)))
    Tags = Concatenate()(tags)
    x = Concatenate(axis=-1)([inputs, inputs2, Tags])

    x = BatchNormalization()(x)
    x = Dropout(0.8)(x)
    for i in range(3):
        x = Dense(256)(x)
        x = BatchNormalization()(x)
        x = Lambda(tf.keras.activations.swish)(x)
        x = Dropout(0.5)(x)

    x = tf.keras.layers.Dense(output_dim)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = Model(inputs=inputs,outputs=out)
    model.compile(
        optimizer=tfa.optimizers.RectifiedAdam(1e-3),
        loss=BinaryCrossentropy(label_smoothing=1e-3),
        # loss="mse",
        metrics=tf.keras.metrics.AUC(name = 'auc')
        )
    return model
v = 4
try:
    os.mkdir(f"models_v{v}")
except:
    pass

In [7]:
import pickle
pickle.dump([features, resp_cols, f_mean, scaler], open(f"models_v{v}/conf.pkl", "wb"))

In [8]:
from sklearn.model_selection import train_test_split

train_indices, test_indices = train_test_split(range(len(X)), test_size=0.1, shuffle=False)

In [ ]:
%%time
SEEDS = [10, 15]

for j, SEED in enumerate(SEEDS):
    set_all_seeds(SEED)

    model = create_model(X.shape[-1],len(resp_cols))
    # X_train, X_test = X[train_indices], X[test_indices]
    # y_train, y_test = y[train_indices], y[test_indices]
    history = model.fit(X[train_indices],y[train_indices],validation_data=(X[test_indices],y[test_indices]),epochs=500,batch_size=4096)

Epoch 1/500
436/436 [==============================] - 56s 76ms/step - loss: 0.7855 - auc: 0.5005 - val_loss: 0.6926 - val_auc: 0.5247
Epoch 2/500
436/436 [==============================] - 32s 74ms/step - loss: 0.6976 - auc: 0.5073 - val_loss: 0.6911 - val_auc: 0.5320
Epoch 3/500
436/436 [==============================] - 32s 73ms/step - loss: 0.6921 - auc: 0.5221 - val_loss: 0.6905 - val_auc: 0.5364
Epoch 4/500
436/436 [==============================] - 31s 72ms/step - loss: 0.6915 - auc: 0.5278 - val_loss: 0.6901 - val_auc: 0.5395
Epoch 5/500
436/436 [==============================] - 31s 72ms/step - loss: 0.6911 - auc: 0.5316 - val_loss: 0.6899 - val_auc: 0.5405
Epoch 6/500
436/436 [==============================] - 31s 72ms/step - loss: 0.6909 - auc: 0.5338 - val_loss: 0.6895 - val_auc: 0.5436
Epoch 7/500
436/436 [==============================] - 31s 71ms/step - loss: 0.6908 - auc: 0.5345 - val_loss: 0.6895 - val_auc: 0.5435
Epoch 8/500
436/436 [==============================] - 

In [ ]:
%%time
SEEDS = [2021, 2022]

for j, SEED in enumerate(SEEDS):
    set_all_seeds(SEED)

    model = create_model(X.shape[-1],len(resp_cols))

    model.fit(X,y,epochs=150,batch_size=4096)

    model.save_weights(f'models_v{v}/model_{SEED}.hdf5')